In [1]:
# import packages

import numpy as np
from numpy import random
from scipy import stats
# import pandas as pd
import copy
import matplotlib.pyplot as plt

# import r packages if needed

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

rpackages.importr("clusterGeneration")
rpackages.importr("mpower")
rpackages.importr("base")
rpackages.importr("utils")
cluster_generation = robjects.packages.importr("clusterGeneration")
mpower = robjects.packages.importr("mpower")
base = robjects.packages.importr("base")
utils = robjects.packages.importr("utils")

# set seed from R
r = robjects.r
set_seed = r('set.seed')

import pickle


In [2]:
# todos
# TODO 1: decide which parameters you include in specific functions (e.g. for beta confounded ex), check
# TODO 2: CLEAN CODE !
# TODO 3: Check if it makes sense to change betaconfoundedness ?

In [3]:
 # Fully Synthetic Data Sets (Set up from Künzel et al.)

# 1: Simulate the d-dimensional X.
# 2: Create Potential Outcomes Y(1) and Y(0).
# 3: Simulate Treatment Assignments trough W.

In [4]:
# TODO: if needed take student_t distributed errors to see simulation with heavy-tailed errors

# e_o = random.standard_t(df=1,size=N)
# e_1 = random.standard_t(df=1,size=N)

In [5]:
# TODO: Chaos here --> Clean

In [6]:
########################################## TODO: HOW TO INCLUDE THESE CONFOUNDED VERSIONS?

In [7]:
''''
# SI6.1 beta confounded, no treatment effect # TODO: CHECK WHETER WHEN CONFOUNDED IT MAKES SENSE TO TAKE ALL BETAS, OR JUST LIKE HERE?
X = random.uniform(low=0, high=1, size=(N, d))  # ACHTUNG: nöd wiederhole!
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0
tau = np.zeros(N)

'''''


"'\n# SI6.1 beta confounded, no treatment effect # TODO: CHECK WHETER WHEN CONFOUNDED IT MAKES SENSE TO TAKE ALL BETAS, OR JUST LIKE HERE?\nX = random.uniform(low=0, high=1, size=(N, d))  # ACHTUNG: nöd wiederhole!\nmu_0 = 2 * X[:, 0] - 1\nmu_1 = mu_0\ntau = np.zeros(N)\n\n"

In [8]:
''''
# SI6.2 beta confounded, simple cate (indicator)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * np.int8(X[:, 1] > 0.4)
tau = mu_1 - mu_0
'''''

"'\n# SI6.2 beta confounded, simple cate (indicator)\nX = random.uniform(low=0, high=1, size=(N, d))\nmu_0 = 2 * X[:, 0] - 1\nmu_1 = mu_0 + 2 * np.int8(X[:, 1] > 0.4)\ntau = mu_1 - mu_0\n"

In [ ]:
''''
# SI6.3 beta confounded, simple cate (linear)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * X[:, 1]
tau = mu_1 - mu_0
'''''

In [ ]:
''''
# SI6.4 beta confounded, simple cate (quadratic)
X = random.uniform(low=0, high=1, size=(N, d))
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 2 * (X[:, 1] ** 2)
tau = mu_1 - mu_0
'''''

In [ ]:
##################################

In [ ]:
''''
# SI7.1 beta confounded, complex cate (linear) # TODO: SAME CHECK FOR CONFOUNDED AND BETAS!
betas = random.uniform(low=-15, high=15, size=d)

mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + np.matmul(X, betas)
tau = mu_1 - mu_0
'''''
# TODO: CHECK WHETHER THIS SETUP MAKES SENSE

In [ ]:
''''
# SI7.2 beta confounded, complex cate (non-linear)
mu_0 = 2 * X[:, 0] - 1
mu_1 = mu_0 + 1 / 2 * varsigma_function(x=X[:, 0]) * varsigma_function(x=X[:, 1])
'''''
# TODO: CHECK WHETER THIS SETUP MAKES SENSE

# FROM SCRATCH

In [9]:
# seeds for reproducibility --> NICE :-D
set_seed(8953)
random.seed(8953)

In [10]:
# SIMULATION SETUP: "META-PARAMETERS"
sample_sizes = [500, 1000]
test_size = 500
n_runs = 2
n_setups = 2
d = 25

In [11]:
# all settings of e_x
exs = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1,
       'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced', 'beta_balanced',
       'beta_balanced',
       'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced', 'beta_unbalanced',
       'beta_unbalanced']

In [12]:
# all settings of mu_0 / mu_1
cates = ['glinear', 'plinear', 'sindcate', 'slinearcate', 'squadcate', 'clinearcate', 'cnonlincate'] * 4

Cate Settings function (unconfounded)

In [13]:
def simple_indicator_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * np.int8(x_train[:, 0] > 0.5)
    mu_1 = mu_0 + 8 * np.int8(x_train[:, 1] > 0.1)  # indicator
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * np.int8(x_test[:, 0] > 0.5)
    mu_1_test = mu_0_test + 8 * np.int8(x_test[:, 1] > 0.1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test


In [14]:
def simple_linear_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * x_train[:, 0]
    mu_1 = mu_0 + 8 * x_train[:, 1]
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * x_test[:, 0]
    mu_1_test = mu_0_test + 8 * x_test[:, 1]
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [15]:
def simple_quadratic_cate(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas) + 5 * (x_train[:, 0] ** 2)
    mu_1 = mu_0 + 8 * (x_train[:, 1] ** 2)
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas) + 5 * (x_test[:, 0] ** 2)
    mu_1_test = mu_0_test + 8 * (x_test[:, 1] ** 2)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [16]:
def complex_linear_cate(x_train, x_test):
    betas_0 = random.uniform(low=-5, high=5, size=x_train.shape[1])
    betas_1 = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas_0)
    mu_1 = np.matmul(x_train, betas_1)
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas_0)
    mu_1_test = np.matmul(x_test, betas_1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [17]:
# helper function
def varsigma_function(x):
    return 2 / (1 + np.exp(-12 * (x - 1 / 2)))

In [18]:
def complex_nonlinear_cate(x_train, x_test):
    # train
    mu_0 = -1 / 2 * varsigma_function(x=x_train[:, 0]) * varsigma_function(x=x_train[:, 1])
    mu_1 = 1 / 2 * varsigma_function(x=x_train[:, 0]) * varsigma_function(x=x_train[:, 1])
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + 1 / 2 * varsigma_function(x=x_test[:, 0]) * varsigma_function(x=x_test[:, 1])
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [19]:
def global_linear_response(x_train, x_test):
    betas = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = np.matmul(x_test, betas)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [20]:
# helper function
def piecewise_beta_helper(dimension):
    betas = np.random.uniform(low=-5, high=5, size=dimension)

    betas_l = copy.copy(betas)  # betas_lower
    betas_l[5:dimension] = 0

    betas_m = copy.copy(betas)  #betas_middle
    betas_m[0:5] = 0
    betas_m[10:dimension] = 0

    betas_u = copy.copy(betas)  #betas_upper
    betas_u[0:10] = 0
    betas_u[15:dimension] = 0

    return betas_l, betas_m, betas_u


In [21]:
# helper function
def piecewise_linear_helper(x, betas_l, betas_u, betas_m):
    condition_l = x[:, 19] < -0.4
    condition_u = x[:, 19] > 0.4

    array = np.zeros(len(x))
    array[condition_l] = np.matmul(x[condition_l, :], betas_l)
    array[~condition_l & ~condition_u] = np.matmul(x[~condition_l & ~condition_u, :], betas_m)
    array[condition_u] = np.matmul(x[condition_u, :], betas_u)

    return array

In [22]:
def piecewise_linear_response(x_train, x_test):
    betas_l, betas_m, betas_u = piecewise_beta_helper(x_train.shape[1])

    # train
    mu_0 = piecewise_linear_helper(x_train, betas_l, betas_u, betas_m)
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = piecewise_linear_helper(x_test, betas_l, betas_u, betas_m)
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

Cate Settings function (confounded)


In [23]:
def confounded_no_treatment_effect(x_train, x_test):
    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0
    tau = np.zeros(len(x_train))

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test
    tau_test = np.zeros(len(x_test))

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test


In [24]:
def confounded_simple_indicator_cate(x_train, x_test):
    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0 + np.int8(x_train[:, 1] > 0.1)  # indicator
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + np.int8(x_test[:, 1] > 0.1)  # indicator
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test


In [25]:
def confounded_simple_linear_cate(x_train, x_test):
    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0 + x_train[:, 1]
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + x_test[:, 1]
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [26]:
def confounded_simple_quadratic_cate(x_train, x_test):
    # train
    mu_0 = 2 * x_train[:, 0] - 1
    mu_1 = mu_0 + (x_train[:, 1] ** 2)
    tau = mu_1 - mu_0

    # test
    mu_0_test = 2 * x_test[:, 0] - 1
    mu_1_test = mu_0_test + (x_test[:, 1] ** 2)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

In [27]:
def confounded_complex_linear_cate(x_train, x_test):
    betas_0 = random.uniform(low=-5, high=5, size=x_train.shape[1])
    betas_1 = random.uniform(low=-5, high=5, size=x_train.shape[1])

    # train
    mu_0 = np.matmul(x_train, betas_0) + 5 * x_train[:, 0]
    mu_1 = np.matmul(x_train, betas_1) + 5 * x_train[:, 0]
    tau = mu_1 - mu_0

    # test
    mu_0_test = np.matmul(x_test, betas_0)
    mu_1_test = np.matmul(x_test, betas_1)
    tau_test = mu_1_test - mu_0_test

    return mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test

ex functions

In [28]:
# helper function
def beta_balanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 4 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 4 * (1 + beta_values_test)

    return e_x, e_x_test


In [29]:
def beta_unbalanced(x_train, x_test):
    beta_dist = stats.beta(a=2, b=4)  # set beta distribution

    # train
    cdf_values = stats.norm.cdf(x_train[:, 0])
    beta_values = beta_dist.pdf(cdf_values)  # calculate pdf values for x1
    e_x = 1 / 40 * (1 + beta_values)

    # test
    cdf_values_test = stats.norm.cdf(x_test[:, 0])
    beta_values_test = beta_dist.pdf(cdf_values_test)  # calculate pdf values for x1
    e_x_test = 1 / 40 * (1 + beta_values_test)

    return e_x, e_x_test

generate train/test function

In [30]:
# TODO: TRIPPLE CHECK THIS FUNCTION, IT IS VERY IMPORTANT
def generate(ex, cate, sample_size, testing_size):
    # 1 generated x_train & x_test
    mean = np.zeros(d)
    cov = np.array(mpower.cvine(d=d, alpha=0.5, beta=0.5))  # TODO: set cov according to setup
    x_train = random.multivariate_normal(mean=mean, cov=cov, size=sample_size, check_valid='warn')
    x_test = random.multivariate_normal(mean=mean, cov=cov, size=test_size, check_valid='warn')

    # 2 generate e_0 & e_0
    # train
    e_0 = random.normal(loc=0.0, scale=1.0, size=sample_size)
    e_1 = random.normal(loc=0.0, scale=1.0, size=sample_size)

    # test
    e_0_test = random.normal(loc=0.0, scale=1.0, size=testing_size)
    e_1_test = random.normal(loc=0.0, scale=1.0, size=testing_size)

    # 3 compute mu_0 & mu_1 --> based on setting
    if cate == 'glinear':  # 'global linear response' setting (no treatment effect)
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = global_linear_response(x_train, x_test)

    elif cate == 'plinear':  # 'piecewise linear response' setting (no treatment effect)
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = piecewise_linear_response(x_train, x_test)

    elif cate == 'sindcate':  # 'simple indicator cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_indicator_cate(x_train, x_test)

    elif cate == 'slinearcate':  # 'simple linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_linear_cate(x_train, x_test)

    elif cate == 'squadcate':  # 'simple quadratic cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = simple_quadratic_cate(x_train, x_test)

    elif cate == 'clinearcate':  # 'complex linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = complex_linear_cate(x_train, x_test)

    elif cate == 'cnonlincate':  # 'complex non-linear cate' setting
        mu_0, mu_1, tau, mu_0_test, mu_1_test, tau_test = complex_nonlinear_cate(x_train, x_test)

    else:
        raise NotImplementedError

    # 4 create potential outcomes Y_0 & Y_1
    # train
    y_0 = mu_0 + e_0
    y_1 = mu_1 + e_1

    # test
    y_0_test = mu_0_test + e_0_test
    y_1_test = mu_1_test + e_1_test

    # 5 SET PROPENSITY SCORE E_X --> based on setting
    if isinstance(ex, float or int):
        e_x = ex
        e_x_test = ex

    elif ex == 'beta_balanced':
        e_x, e_x_test = beta_balanced(x_train, x_test)

    elif ex == 'beta_unbalanced':
        e_x, e_x_test = beta_unbalanced(x_train, x_test)

    else:
        raise NotImplementedError

    # 6 generate treatment assignment  W
    w = random.binomial(size=sample_size, n=1, p=e_x)
    w_test = random.binomial(size=testing_size, n=1, p=e_x_test)

    # 7 create observed variables Y
    y = np.multiply(w, y_1) + np.multiply(np.ones(sample_size) - w, y_0)
    y_test = np.multiply(w_test, y_1_test) + np.multiply(np.ones(testing_size) - w_test, y_0_test)

    # 8 create train & test sets
    dataset_train = np.concatenate(
        (np.reshape(y, (sample_size, 1)), x_train, np.reshape(w, (sample_size, 1)), np.reshape(tau, (sample_size, 1))),
        axis=1)
    dataset_test = np.concatenate((np.reshape(y_test, (testing_size, 1)), x_test, np.reshape(w_test, (testing_size, 1)),
                                   np.reshape(tau_test, (testing_size, 1))), axis=1)

    # 9 return both sets
    return [dataset_train, dataset_test]


empty list

In [31]:
# create empty list to save all datasets
data = []
for i in range(n_setups):
    data.append([])

for i in range(n_setups):
    for s in range(4):
        data[i].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            data[i][s].append([])

for i in range(n_setups):
    for s in range(4):
        for r in range(n_runs):
            for t in range(2):
                data[i][s][r].append([])

generate all

In [32]:
# generate all datasets
s = 0
for setup in range(n_setups):
    print(f'Generating setup {setup + 1}')
    for size in sample_sizes:
        for run in range(n_runs):
            gen = generate(ex=exs[setup], cate=cates[setup], sample_size=size, testing_size=test_size)
            data[setup][s][run][0] = gen[0]  # add train-set
            data[setup][s][run][1] = gen[1]  # add test-set
        s += 1  # update index for sample_sizes
    s = 0  # reset index for sample_sizes
print('DONE')
# it took 43 seconds # 4.4 GB of data!

Generating setup 1
Generating setup 2
DONE


In [33]:
data[0][0][0][0]  # :)

array([[-24.3859079 ,   0.76131199,   0.07103195, ...,   1.41694044,
          1.        ,   0.        ],
       [-28.59538751,  -1.52617665,   1.57529871, ...,  -0.84022404,
          0.        ,   0.        ],
       [ 12.59817261,   0.28280631,  -0.50569088, ...,   0.40840675,
          1.        ,   0.        ],
       ...,
       [  8.23552418,   0.79543398,  -0.22851133, ...,  -0.2908772 ,
          1.        ,   0.        ],
       [  5.03496137,   0.03192942,  -0.29086033, ...,   0.87846121,
          1.        ,   0.        ],
       [ 17.56439986,  -0.27769541,  -0.3103303 , ...,  -0.7269873 ,
          0.        ,   0.        ]])

In [ ]:
# -1.992791, -7.838958 -> reproducibility check over :)

Just Plotting relationship

In [ ]:
x_points = data[22][3][0][0][:, 21]
y_points = data[22][3][0][0][:, 0]
plt.plot(x_points, y_points, 'o')
plt.show()

# cant see any relationship
# probably to small confounding due too betas!

# Save data with pickle

In [35]:
# save data
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic_500_1000.pkl"

open_file = open(file_name, "wb")
pickle.dump(data, open_file)
open_file.close()

In [36]:
file_name = "/Users/arberimbibaj/Documents/Master Thesis ETH/DataSets /Generated/data_synthetic_500_1000.pkl"
open_file = open(file_name, "rb")
loaded_data = pickle.load(open_file)
open_file.close()

loaded_data[0][0][0][0]

array([[-24.3859079 ,   0.76131199,   0.07103195, ...,   1.41694044,
          1.        ,   0.        ],
       [-28.59538751,  -1.52617665,   1.57529871, ...,  -0.84022404,
          0.        ,   0.        ],
       [ 12.59817261,   0.28280631,  -0.50569088, ...,   0.40840675,
          1.        ,   0.        ],
       ...,
       [  8.23552418,   0.79543398,  -0.22851133, ...,  -0.2908772 ,
          1.        ,   0.        ],
       [  5.03496137,   0.03192942,  -0.29086033, ...,   0.87846121,
          1.        ,   0.        ],
       [ 17.56439986,  -0.27769541,  -0.3103303 , ...,  -0.7269873 ,
          0.        ,   0.        ]])

In [ ]:
# TODO: CHANGE THIS BACK TO WHAT IT WAS, SO NO CRASHES CAN HAPPEN.
# Current values:
# NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
# NotebookApp.rate_limit_window=3.0 (secs)